*Finally, train a Bloom Filter that filter out bots from the stream.*
- *Find the correct parameters for the bloom filter having an error around 10%*

Bloom filter is characterized by:
- size of a bit array ($m$)
- number of hash functions

where optimal $m$ can be calculated as:
$$m=-\frac{n*ln(p)}{ln(2)^{2}}$$
,where n is a number of items (bots in our case)\
and number of hash functions as:
$$k=\frac{m}{n}ln(2)$$

So, for the training to start we need to evaluate a number of bots from our sample.

In [ ]:
import pandas as pd

def count_true_rows(file_path: str) -> int:
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Count rows where 'flag' is True
    count_true = df['bot'].sum() if 'bot' in df else 0

    return count_true

In [63]:
import numpy as np

p = 0.1  # 10% error rate

n = count_true_rows('ucu-mmds/src/homework2/sampled_events_id.csv')
m = - n * np.log(p) / np.log(2)**2
k = np.log(2) * m / n

print(f"for n={n} array size (m) is {m}, k is {k}")

n = count_true_rows('ucu-mmds/src/homework2/sampled_events_timestamp.csv')
m = - n * np.log(p) / np.log(2)**2
k = np.log(2) * m / n

print(f"for n={n} array size (m) is {m}, k is {k}")

for n=9361 array size (m) is 44862.86573526829, k is 3.3219280948873617
for n=9138 array size (m) is 43794.13172619182, k is 3.3219280948873617


So, ~44kb array and 3 hash functions are sufficient to store a bloom filter trained with 40k samples.

In [65]:
import hashlib

def hashes(value, num_hashes, size):
    h1 =  int(hashlib.md5(value.encode()).hexdigest(), 16) % size
    h2 =  int(hashlib.sha256(value.encode()).hexdigest(), 16) % size
    return [(h1 + i * h2) % size for i in range(num_hashes)]

class BloomFilter:
    def __init__(self, size: int, hash_count: int):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = [0] * size

    def add(self, item: str):
        for index in hashes(item, self.hash_count, self.size):
            self.bit_array[index] = True

    def add_all(self, items: list):
        for item in items:
            self.add(item)

    def contains(self, item: str) -> bool:
        for index in hashes(item, self.hash_count, self.size):
            if not self.bit_array[index]:
                return False
        return True


In [68]:
df = pd.read_csv('ucu-mmds/src/homework2/sampled_events_id.csv')

filter = BloomFilter(int(m), int(k)+1)
filter.add_all(df['user'].to_list())

In [76]:
print(filter.contains(df['user'][3]))  # True
print(filter.contains('$@@##'))  # True

True
False
